In [1]:
import sys, os
import matplotlib.pyplot as plt
os.chdir('/home/cfchen/Github/darknet3/')

In [2]:
ls -l

total 249156
drwxrwxr-x 2 cfchen cfchen      4096  四  24 14:24 cfg/
-rwxrwxr-x 1 cfchen cfchen   1933248  五   7 15:49 darknet*
drwxrwxr-x 3 cfchen cfchen      4096  四  24 14:24 data/
drwxrwxr-x 3 cfchen cfchen      4096  五   9 17:42 examples/
drwxrwxr-x 2 cfchen cfchen      4096  四  24 14:24 include/
-rw-rw-r-- 1 cfchen cfchen   2141966  五   7 15:49 libdarknet.a
-rwxrwxr-x 1 cfchen cfchen   1854984  五   7 15:49 libdarknet.so*
-rw-rw-r-- 1 cfchen cfchen       515  四  24 14:24 LICENSE
-rw-rw-r-- 1 cfchen cfchen       474  四  24 14:24 LICENSE.fuck
-rw-rw-r-- 1 cfchen cfchen      6652  四  24 14:24 LICENSE.gen
-rw-rw-r-- 1 cfchen cfchen     35141  四  24 14:24 LICENSE.gpl
-rw-rw-r-- 1 cfchen cfchen       360  四  24 14:24 LICENSE.meta
-rw-rw-r-- 1 cfchen cfchen      1071  四  24 14:24 LICENSE.mit
-rw-rw-r-- 1 cfchen cfchen       461  四  24 14:24 LICENSE.v1
-rw-rw-r-- 1 cfchen cfchen      2906  四  25 08:38 Makefile
drwxrwxr-x 2 cfchen cfchen      4096  五   7 15:49 obj/
-rw-rw-r-- 1 cfchen cfche

In [3]:
import python.darknet as dn
#import pdb

## Test

In [4]:
dn.set_gpu(0)

net = dn.load_net(str.encode("cfg/yolov3.cfg"),
                  str.encode("yolov3.weights"), 0)

meta = dn.load_meta(str.encode("cfg/coco.data"))

r = dn.detect(net, meta, str.encode("data/dog.jpg"))
print(r)

[(b'bicycle', 0.9941930770874023, (363.78155517578125, 278.9617004394531, 397.52557373046875, 331.16278076171875)), (b'dog', 0.990459680557251, (221.56259155273438, 380.4324645996094, 186.33229064941406, 312.1522216796875)), (b'truck', 0.911002516746521, (581.749267578125, 127.70828247070312, 215.17501831054688, 86.23703002929688))]


In [5]:
r = dn.detect(net, meta, str.encode("data/eagle.jpg"))
print(r)
print("")

r = dn.detect(net, meta, str.encode("data/giraffe.jpg"))
print(r)
print("")

r = dn.detect(net, meta, str.encode("data/horses.jpg"))
print(r)
print("")

r = dn.detect(net, meta, str.encode("data/person.jpg"))
print(r)

[(b'bird', 0.9960672855377197, (369.8246765136719, 263.11767578125, 476.1756591796875, 363.6462707519531))]

[(b'giraffe', 0.9977366328239441, (296.2441711425781, 241.77305603027344, 288.03546142578125, 384.6851806640625)), (b'zebra', 0.9433543086051941, (339.7536926269531, 325.0368347167969, 180.08058166503906, 247.6002197265625))]

[(b'horse', 0.9759820699691772, (168.1838836669922, 306.0086364746094, 321.7207336425781, 234.34136962890625)), (b'horse', 0.9714113473892212, (339.3211975097656, 271.762939453125, 223.04824829101562, 178.17758178710938)), (b'horse', 0.9103319644927979, (518.2222900390625, 275.6620788574219, 161.98268127441406, 139.36647033691406)), (b'horse', 0.8900034427642822, (81.26667785644531, 273.5216064453125, 153.34776306152344, 195.1179656982422))]

[(b'person', 0.9998968839645386, (233.6996307373047, 232.0786895751953, 85.39936065673828, 275.8518371582031)), (b'horse', 0.9968202710151672, (498.75482177734375, 243.34120178222656, 208.79769897460938, 215.735519409

## Functions

In [6]:
from scipy.misc import imread

def array_to_image(arr):
    arr = arr.transpose(2,0,1)
    c = arr.shape[0]
    h = arr.shape[1]
    w = arr.shape[2]
    arr = (arr/255.0).flatten()
    data = dn.c_array(dn.c_float, arr)
    im = dn.IMAGE(w,h,c,data)
    return im

In [7]:
import cv2
import numpy as np
import colorsys
import time
from time import time as timer

cv2.__version__

'3.4.0'

## Video

In [10]:
count_frame, process_every_n_frame = 0, 1

# get video
capture = cv2.VideoCapture('/media/cfchen/956df7bc-562e-4f24-8339-fd0b67f98888/Downloaded/VideosHPB/IMAG0011.mp4')

fourcc = cv2.VideoWriter_fourcc(*'XVID')
#out = cv2.VideoWriter('yolo3.avi',fourcc, 30.0, (1920, 1080))
out = cv2.VideoWriter('yolo3.avi',fourcc, 30.0, (960, 540))     # resize image half

print("Video Dim: {}  {}  {}".format(capture.get(3), 'X', capture.get(4)))

Video Dim: 1920.0  X  1080.0


In [9]:
while(True):
    # get a frame
    ret, frame = capture.read()
    
    if ret == True:
        count_frame += 1
    
        # show a frame
        img = cv2.resize(frame, (0, 0), fx=0.5, fy=0.5)  # resize image half
        #print(img.shape)
        out.write(img) 
        cv2.imshow("Video", img)
    
        if (cv2.waitKey(1) & 0xFF == ord('q')) or (ret == False):
            break
            
    else:
        break
            
print('++++++++ Video End ++++++++')

capture.release()
out.release()
cv2.destroyAllWindows()   

++++++++ Video End ++++++++


## Bounding Box

In [12]:
while(True):
    # get a frame
    ret, frame = capture.read()
    
    if ret == True:
        count_frame += 1
    
        # show a frame
        img = cv2.resize(frame, (0, 0), fx=0.5, fy=0.5)  # resize image half
        #print(img.shape)
        out.write(img) 
        cv2.imshow("Video", img)
        
        if count_frame % process_every_n_frame == 0:
            cv2.imshow("YOLO", pipeline(img))

    
        if (cv2.waitKey(1) & 0xFF == ord('q')) or (ret == False):
            break
            
    else:
        break
            
print('++++++++ Video End ++++++++')

capture.release()
out.release()
cv2.destroyAllWindows()   

ArgumentError: argument 1: <class 'TypeError'>: wrong type

In [11]:
def pipeline(img):
    # image data transform：
    # img - cv image
    # im - yolo image
    im = array_to_image(img)
    dn.rgbgr_image(im)

    result = dn.detect(net, meta, im)
    print(result)

    img_final = draw_boxes(img, result)
    return img_final

In [18]:
'''
import sys, os

ROOT_DIR = os.path.abspath("../python/")
    
#sys.path.append(ROOT_DIR)  
sys.path.append(os.path.join(os.getcwd(),'python/'))

print(ROOT_DIR)
'''

'\nimport sys, os\n\nROOT_DIR = os.path.abspath("../python/")\n    \n#sys.path.append(ROOT_DIR)  \nsys.path.append(os.path.join(os.getcwd(),\'python/\'))\n\nprint(ROOT_DIR)\n'